# output metric

In [1]:
output_metric = {
                "Total CO2e Emissions":
                    ["Emissions?"],
                "CO2e Emissions, Scope 1":
                    ["Emissions?", "Scope 1"],
                "CO2e Emissions, Scope 2":
                    ["Emissions?", "Scope 2"],
                "CO2e Emissions, Scope 3":
                    ["Emissions?", "Scope 3"],
                "Emissions Policy":
                    ["Emissions?"],
                "Emission Reduction Target %":
                    ["Emissions?"],
                "Emissions Reduction Target Year":
                    ["Emissions?"],
                "Total Water Withdrawal":
                    ["Water"],
                "Water Discharged":
                    ["Water"],
                "Total Fresh Water Withdrawal":
                    ["Water"],
                "Water Recycled":
                    ["Water"],
                "Water Efficiency Target":
                    ["Water"],
                "Total Energy Consumed":
                    ["Energy"],
                "Total Renewable Energy":
                    ["Energy"],
                "Renewable Energy Consumed":
                    ["Energy"],
                "Renewable Energy Use":
                    ["Energy"],
                "Total Waste":
                    ["Waste"],
                "Women Executives %":
                    ["Women", "Female"],
                "Women Board Members %":
                    ["Women", "Female"],
                "Total Board Members":
                    ["Board Members?"],
                "Independent Board Members %":
                    ["Board Members?"],
                "Audit Committee Independence %":
                    ["Committee Independence"],
                "Compensation Committee Independence %":
                    ["Committee Independence"],
                "Nomination Committee Independence %":
                    ["Committee Independence"],
                "ESG Sustainability Reporting":
                    ["Sustainability Reporting", "ESG reporting"],
                "Lost Time Injury Rate - Employees":
                    ["Injury", "Emlployees?"],
                "Avg Training Hours / Employee":
                    ["Training Hours?", "Employee"],
                "Trade Union Percentage":
                    ["Trade Union"],
                "CEO Salary / Average Salary":
                    ["Salary", "CEO Salary", "Average Salary"],
                "Employee Turnover %":
                    ["Employee", "Turnover"],
                "Involuntary Employee Turnover Rate":
                    ["Employee", "Turnover"],
                "Women's / Men's Compensation %":
                    ["Women", "Female", "Men", "Male"],
                "Women Employees %":
                    ["Women", "Female", "Employees?"],
                "Women Managers %":
                    ["Women", "Female", "Managers?"],
                "Minority Employees %":
                    ["Minority", "Employees?"]
                }

In [2]:
# key_words = ["Emissions?",
#              "Scope [0-3]",
#              "Water",
#              "Energy",
#              "Waste",
#              "Women",
#              "Female"
#              "Board Members?",
#              "Board Committee",
#              "Committee Independence",
#              "sustainability report",
#              "ESG report",
#              "Employees?",
#              "Injury",
#              "Training Hours",
#              "Salary",
#              ]
# position_record = {key_word:[] for key_word in key_words}

# tables

In [3]:
import camelot
import os
def parseTable_camelot(input_path, file, output_path, pages = "all"):
    print("parsing table from " + file + "at pages " + pages)
    try:
        tables = camelot.read_pdf(os.path.join(input_path, file), pages=pages)
        if len(tables) > 0:
            print("check the information")
            for table in tables:
                # print(table.cols)
                # print(table.rows)
                # print(tabel.accuracy)
                table_df = table.df
                columns = table_df.shape[1]
                for key_word,value_words  in output_metric.items():
                    for col in range(columns):
                        contains = any(table.df[col].str.contains("|".join(value_words), case=False, regex=True).tolist())
                        if contains:
                            position_record[key_word].append(str(table.page))
                            break
            # print("saving {len(tables)} tables")
            # accuracy = [str(table.parsing_report["accuracy"]) for table in tables]
            # print("accuracy list " + " ".join(accuracy))
            # tables.export(os.path.join(output_path, file.replace("pdf", "xlsx")), f='excel')
        else:
            print(f"no tables found for " + file)
    except Exception as e:
        print("error parsing table from " + file)    
        print(e)      

In [4]:
import tabula
import pandas as pd
def parseTable_tabula(input_path, file, output_path, pages):
    print("parsing table from " + file + "at pages" + pages)
    try:
        tables = tabula.read_pdf(os.path.join(input_path, file), pages=pages, lattice=True)
        if len(tables) > 0:
            print("saving {len(tables)} tables")
            with pd.ExcelWriter(os.path.join(output_path, file.replace("pdf", "xlsx"))) as f_obj:
                for i,table in enumerate(tables):
                    table.to_excel(f_obj, sheet_name=str(i))
        else:
            print(f"no tables found for " + file)
    except Exception as e:
        print("error parsing table from " + file)    
        print(e) 


# texts

In [5]:
import re
def check_information(page, text):
    for key_word, value_word in output_metric.items():
        pattern = "|".join(value_word)
        rex = re.search(pattern, text, re.I)
        if rex != None:
            position_record_text[key_word].append(str(page))

In [6]:
import PyPDF2
def parseText(input_path, file, output_path, pages):
    print("parsing text from " + file + " at pages " + pages)
    object = PyPDF2.PdfFileReader(os.path.join(input_path, file))
    num_pages = object.getNumPages()
    for i in range(0, num_pages):
        page = object.getPage(i)
        text = page.extractText()
        text = text.replace('\n','')
        check_information(i, text)
        # with open(os.path.join(output_path, file.replace("pdf", "text")), "a") as f_obj:
        #     f_obj.write(text)

# main function

In [7]:
import os
import pandas as pd

input_path = "./pdf_files"
output_table_camelot = "./table_files_camelot"
output_table_tabula = "./table_files_tabula"
output_path_text = "./text_files"
pdf_files = os.listdir(input_path)
name = ["output_metric", "page"]
for file in pdf_files:
    # 1 parsing the table and text
    position_record_table = {key_word:[] for key_word in output_metric.keys()}
    position_record_text = {key_word:[] for key_word in output_metric.keys()}
    # parseTable_camelot(input_path, file, output_table_camelot, pages="all")
    # parseTable_tabula(input_path, file, output_table_tabula)
    parseText(input_path, file, output_path_text, "all")
    # 2 save the result
    position_record = {key_word:position_record_table[key_word]+position_record_text[key_word] 
                       for key_word in output_metric.keys()}
    position_record = {key_word:list(set(pages)) for key_word, pages in position_record.items()}
    position_matchs = [[key, " ".join(value)] for key, value in position_record.items()]
    data = pd.DataFrame(columns=name, data=position_matchs)
    data.to_excel("./data_extraction/" + file.replace(".pdf", "_text.xlsx"))

parsing text from 2021-tesla-impact-report.pdfat pages all
